In [98]:
import requests
import re
from bs4 import BeautifulSoup
import csv
import os

#methodology; TLDR: idempotent write op with job-id as primary key
# 1. use cvs as database
# 2. load csv in cache
# 3. scrape & parse linkedin jobs
# 4. look up jobid, omit if exists in cache, add to cache and jobs_to_add array
# 5. write jobs_to_add back to csv

In [99]:
#configurations
base_url_prefix = "https://www.linkedin.com/jobs/search?keywords=software%20engineer%20OR%20engineering%20manager&location=United%20States&pageNum=0&start="
row_increment_default = 25
max_row_default = 51
debug_mode = False
debug_company=""
csv_name="linkedin-job-scraper-database.csv"
csv_columns=['job_id','company','job_type','title','location','link','date']

In [100]:
#initializations
jobs={} # updated cache, won't be written back to csv
jobs_to_add=[] # write-back as additions, strictly as a write buffer

In [101]:
def load_csv_in_cache(reader):
    for row in reader: #reader knows first row is headers
        key = row['job_id']
        value = {k: v for k, v in row.items() if k != 'job_id'}
        jobs[key] = value

    print(f"{len(jobs)} rows read from csv")
    return

In [102]:
# load csv into cache
if not os.path.exists(csv_name):
    with open(csv_name, 'w', newline='') as csvfile:
        writer = csv.writer(csvfile) #this will create new file if not exist
        writer.writerow(csv_columns)
else:
    with open(csv_name, 'r+', newline='') as csvfile:
        writer = csv.writer(csvfile)
        reader = csv.DictReader(csvfile)
        load_csv_in_cache(reader)

94 rows read from csv


In [103]:
for i in range(0,max_row_default,row_increment_default):
    base_url = f"{base_url_prefix}{i}"
    response = requests.get(base_url)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        job_listings = soup.find_all('div', {'class':'job-search-card'})
        for job in job_listings:
            try:
                date = job.find('time', {'class':'job-search-card__listdate'}).attrs['datetime']
            except AttributeError:
                date = job.find('time', {'class':'job-search-card__listdate--new'}).attrs['datetime']
            
            title = job.find('h3', {'class': 'base-search-card__title'}).text.strip()
            job_type = 'em' if 'manager' in title.lower() else 'eng'
            company = job.find('a', {'class': 'hidden-nested-link'}).text.strip()
            location = job.find('span', {'class': 'job-search-card__location'}).text.strip()
            link = job.find('a', {'class': 'base-card__full-link'}).attrs['href']
            pattern = r"(.*-)(\d+)"
            job_id = re.search(pattern,link).group(2)

            #debug
            if debug_mode and debug_company != "" and company == debug_company:
                print(f"id:{job_id},title:{title},job_type:{job_type},link:{link}")
            
            #look up job_id, omit if cache hit, add to cache and jobs_to_add array if cache miss
            if jobs.get(job_id,'') != '': #cache hit
                if debug_mode:
                    print(f"we've seen {company}-{job_id}")
                continue
            else:                    
                jobs.setdefault(job_id,{
                    csv_columns[1]: company,
                    csv_columns[2]: job_type,
                    csv_columns[3]: title,
                    csv_columns[4]: location,
                    csv_columns[5]: link,
                    csv_columns[6]: date
                })
                jobs_to_add.append([job_id,company,job_type,title,location,link,date])
                
    else:
        print("Failed to fetch job listings.")
        break

we've seen The Walt Disney Company-3929518602
we've seen Nike-3925757014
we've seen Nike-3928633016
we've seen ECP-3929025692
we've seen Flint-3929987470
we've seen Kindo-3928613760
we've seen Lockheed Martin-3926404198
we've seen Lockheed Martin-3931026727
we've seen Shopmonkey-3930814205
we've seen Interclypse-3929257489
we've seen Mayo Clinic-3927682458
we've seen Netflix-3931040226
we've seen Trilogy International-3917539672
we've seen LinkedIn-3921122178
we've seen Netflix-3922244352
we've seen iSqFt-3922090249
we've seen Lumafield-3931029784
we've seen Ever-3927302676
we've seen Airbnb-3927279164
we've seen LinkedIn-3925338431
we've seen Ekho (YC S22)-3927314395
we've seen Tripalink-3919967332
we've seen Diesel Laptops-3930095568
we've seen SHEIN Technology LLC-3930836493
we've seen Amicus-3924707625
we've seen Hertz-3916825101
we've seen Backbone-3926154951
we've seen Charles Schwab-3927384721
we've seen Mayo Clinic School of Health Sciences-3928296915
we've seen Tagup, Inc.-392

In [104]:
with open(csv_name, 'a', newline='') as csvfile:
    writer = csv.writer(csvfile)

    # Append all new rows
    writer.writerows(jobs_to_add)
    print(f"{len(jobs_to_add)} jobs added")
jobs_to_add=[] #reset

0 jobs added


In [ ]:
#todo connect to google drive via api